## Authors
* Martin Wegmann (martin.wegmann@unibe.ch)


The goal of this notebook is to show the code behind some of the figures in the manuscript. This notebook focuses mostly on the 2m temperature plots only, but the SLP plots follow the same logic and code structure. Note that a final finish/touch on some of the plots in the manuscript was performed in Adobe Illustrator, such as pasting pdfs together, changing font size and style etc. 

# load libraries

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import cm
import cartopy.crs as ccrs
import matplotlib.pyplot as plt 
import os
import scipy
import matplotlib.colors
from matplotlib.colors import BoundaryNorm
import cdo
CDO = cdo.Cdo
import netCDF4
import datetime
from mycolorpy import colorlist as mcp
import xskillscore as xs
import seaborn as sns
from matplotlib import rcParams
rcParams['font.family'] = 'Arial'
from pandas import Timestamp
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Tahoma']
import iris
from eofs.iris import Eof


# folders and functions

In [ ]:
folder_ML_fields="/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields/"

In [ ]:
folder_asf_fields="/Volumes/SPARK/CNN_seaspredc_analysis/asf20c_fields_postprocessed/"

In [ ]:
folder_ML_fields_postprocessed="/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/"

In [ ]:
folder_20crv3_fields_postprocessed="/Volumes/SPARK/CNN_seaspredc_analysis/20crv3_fields/"

In [ ]:
folder_20crv3_fields="/Volumes/SPARK/20crv3/"

In [ ]:
folder_ekf400_fields="/Volumes/SPARK/ekf400v2/ensmean/"

In [ ]:
folder_corr_fields="/Volumes/SPARK/CNN_seaspredc_analysis/corr_fields/"

In [ ]:
tcr_folder=folder_20crv3_fields_postprocessed
era20c_folder="/Volumes/SPARK/CNN_seaspredc_analysis/era20c_fields/"
prediction_folder=folder_ML_fields_postprocessed
asf20c_folder=folder_asf_fields
output_folder="/Volumes/SPARK/CNN_seaspredc_analysis/analysis_output/plots/"

In [ ]:
preface=["best_modelanomnorm_mpicodaearlyctlrcp26_may_sst_","best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_","best_modelanomnorm_mpicodaearlyctlrcp26_may_sst_","best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_"]

In [ ]:
variables=["t2m_","slp_"]
seasons=["jun_","jul_","aug_","JJA_","dec_","jan_","feb_","DJF_"]


In [ ]:
methods_4_publish=["reg_","CNN8_","CNN11kiri_","CNN44_","unet1_small_","unet4_small_"]
methods_paper=["reg_","auto1_","CNN7_","CNN8_","CNN11kiri_","CNN44_","RNN1_","unet1_small_","unet4_small_"]

methods_paper=["reg_","CNN8_","CNN11kiri_","CNN44_","unet1_small_","unet4_small_"]




In [ ]:
variables_20cr=["air.2m._","prmsl_"]

In [ ]:
month_dict = {
  "jun_": "19002015",
  "jul_": "19002015",
  "aug_": "19002015",
    "JJA_": "19002015",
  "dec_": "19002014",
  "jan_": "19012015",
      "feb_": "19012015",
  "DJF_": "19012015",}

In [ ]:
month_init_dict = {
  "jun": "may",
  "jul": "may",
  "aug": "may",
    "JJA": "may",
  "dec": "nov",
  "jan": "nov",
      "feb": "nov",
  "DJF": "nov"}

In [ ]:
current_cmap = cm.get_cmap("seismic")
current_cmap.set_bad(color='gray')

In [ ]:
def corr_2d_sign(data1,data2,var1,var2,lon1,lat1,save_folder,outname,p=0.05):

    
    # ---------------------
    xr.set_options(keep_attrs=True)
    import numpy as np
    import datetime
    from netCDF4 import Dataset,num2date,date2num
    # -----------------------


    data1_var=data1[var1]
    data2_var=data2[var2]

    latitudes=data1_var[lat1]
    longitudes=data1_var[lon1]
    
    if data1_var.shape==data2_var.shape:
        print("Same dims")
    else:
        print("Not same dims")
    timesteps=data1_var.shape[0]
    data1_var_4_corr=data1_var.values.reshape(timesteps,data1_var.shape[2]*data1_var.shape[1])
    data2_var_4_corr=data2_var.values.reshape(timesteps,data2_var.shape[2]*data2_var.shape[1])
    structure_R= np.arange(data1_var_4_corr.shape[1], dtype=float)
    structure_R_masked= np.arange(data1_var_4_corr.shape[1], dtype=float)
    structure_p= np.arange(data1_var_4_corr.shape[1], dtype=float)

    for a in range(0,data2_var_4_corr.shape[1]):
        one_R=stats.pearsonr(data2_var_4_corr[:,a], data1_var_4_corr[:,a])
        structure_R[a]=one_R[0]
        if one_R[1] <= p:
            structure_R_masked[a]=one_R[0]
        else:
            structure_R_masked[a] = np.ma.masked_greater(2, p)
        structure_p[a]=one_R[1]
    corr_matrix=structure_R.reshape(data1_var.shape[1],data1_var.shape[2])
    corr_matrix_masked=structure_R_masked.reshape(data1_var.shape[1],data1_var.shape[2])
    p_matrix_rnn_ekf400_DJF=structure_p.reshape(data1_var.shape[1],data1_var.shape[2])

    print(structure_R.mean())
    #fig = plt.figure(figsize=(10, 5))
    #plt.imshow(corr_matrix_masked, vmin=-1, vmax=1, cmap="seismic",origin='upper',interpolation="none") 
    #plt.colorbar()
    #plt.show()
    
        
    nyears = 1;
    output=os.path.join(save_folder, outname+".nc")
    unout = 'days since 1900-01-01 00:00:00'
    # -----------------------
    ny, nx = (corr_matrix_masked.shape[0], corr_matrix_masked.shape[1])
    lon = longitudes
    lat = latitudes
    
    dataout = corr_matrix_masked; # create some random data
    datesout = [datetime.datetime(1900+iyear,1,1) for iyear in range(nyears)]; # create datevalues
    # =========================
    ncout = Dataset(output,'w','NETCDF4'); # using netCDF3 for output format 
    ncout.createDimension('lon',nx);
    ncout.createDimension('lat',ny);
    ncout.createDimension('time',nyears);
    lonvar = ncout.createVariable('lon','float32',('lon'));lonvar[:] = lon;
    latvar = ncout.createVariable('lat','float32',('lat'));latvar[:] = lat;
    timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=date2num(datesout,unout);
    myvar = ncout.createVariable("corr",'float32',('time','lat','lon'));myvar.setncattr('units',"R");myvar[:] = dataout;
    ncout.close();

    final_file=xr.open_dataset(output)
    return final_file

# processing T2m data

In [ ]:
# era 20c
# this for 2m temperatures

correlation_afs_time_list_LN=[]
correlation_afs_time_list_EN=[]
correlation_ml_time_list_LN=[]
correlation_ml_time_list_EN=[]

correlation_afs_space_list=[]
correlation_afs_time_list=[]
rmse_afs_space_list=[]
rmse_afs_time_list=[]
p_afs_space_list=[]
p_afs_time_list=[]


correlation_ml_space_list=[]
correlation_ml_time_list=[]
rmse_ml_space_list=[]
rmse_ml_time_list=[]
p_ml_space_list=[]
p_ml_time_list=[]

naming_list=[]

correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]


for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]
    
    print(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")

    start_year=str(afs.forecast_time0.values[0])[0:4]
    print(start_year)
    
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    print(end_year)
    
    print(era20c_folder+"era20c_t2m_"+season+"_lowlow_anoms.nc")
    trcr=xr.open_dataset(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    trcr_time_long=trcr.time
    
    era20c=xr.open_dataset(era20c_folder+"era20c_t2m_"+season+"_lowlow_anoms.nc")
    era20c=era20c.sel(time=slice(start_year,end_year))
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'2T_GDS0_SFC': 't2m'})
    
    afs["time"]=era20c.time
    
    era20c_LN=era20c.sel(time=LN_nov_years_np,method="nearest")
    era20c_EN=era20c.sel(time=EN_nov_years_np,method="nearest")

    
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    
    r_afs_time = xs.pearson_r(afs["t2m"], era20c["t2m"], dim='time')
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["t2m"], era20c_LN["t2m"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["t2m"], era20c_EN["t2m"], dim='time')
        correlation_afs_time_list_LN.append(r_afs_time_LN)
        correlation_afs_time_list_EN.append(r_afs_time_EN)
    r_afs_space = xs.pearson_r(afs["t2m"], era20c["t2m"], dim=["lat", "lon"])
    rmse_afs_time = xs.rmse(afs["t2m"], era20c["t2m"], dim='time')
    rmse_afs_space = xs.rmse(afs["t2m"], era20c["t2m"], dim=["lat", "lon"])
    p_afs_time = xs.pearson_r_p_value(afs["t2m"], era20c["t2m"], dim="time")
    p_afs_space = xs.pearson_r_p_value(afs["t2m"], era20c["t2m"], dim=["lat", "lon"])
    correlation_afs_space_list.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)

    correlation_masked_afs_time_list=[]
    for member_index in np.arange(0,51):
        afs_single_member=afs.sel(member=member_index)
        outname="corr_afs_era20c_t2m_"+season+"_"+str(member_index)
        corr_file=corr_2d_sign(data1=afs_single_member,data2=era20c,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        correlation_masked_afs_time_list.append(corr_file)
    concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    concat_mean=concat.mean(dim="member")
    concat_min=concat.min(dim="member")
    concat_max=concat.max(dim="member")
    correlation_masked_afs_time_list_season_max.append(concat_max)
    correlation_masked_afs_time_list_season_mean.append(concat_mean)
    correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=trcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")

        #print(ml["t2m"].time.values)
        #print(era20c["t2m"].time.values)
        r_ml_time = xs.pearson_r(ml["t2m"], era20c["t2m"], dim='time')
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["t2m"], era20c_LN["t2m"], dim='time')
            r_ml_time_EN = xs.pearson_r(ml_EN["t2m"], era20c_EN["t2m"], dim='time')
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
        
        r_ml_space = xs.pearson_r(ml["t2m"], era20c["t2m"], dim=["lat", "lon"])
        rmse_ml_time = xs.rmse(ml["t2m"], era20c["t2m"], dim='time')
        rmse_ml_space = xs.rmse(ml["t2m"], era20c["t2m"], dim=["lat", "lon"])
        p_ml_time = xs.pearson_r_p_value(ml["t2m"], era20c["t2m"], dim="time")
        p_ml_space = xs.pearson_r_p_value(ml["t2m"], era20c["t2m"], dim=["lat", "lon"])
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)
        
        correlation_masked_ml_time_list=[]
        for member_index in np.arange(0,5):
            outname="corr_"+method+"era20c_t2m_"+season+"_"+str(member_index)
            ml_single_member=ml.sel(member=member_index)
            corr_file=corr_2d_sign(data1=ml_single_member,data2=era20c,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
            correlation_masked_ml_time_list.append(corr_file)
        concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        concat_mean=concat.mean(dim="member")
        
        correlation_masked_ml_time_list_method_mean.append(concat_mean)
    

    

    
    correlation_ml_space_list.append(correlation_ml_space_list_methods)
    correlation_ml_time_list.append(correlation_ml_time_list_methods)
    if season=="DJF":
        correlation_ml_time_list_EN.append(correlation_ml_time_list_methods_EN)
        correlation_ml_time_list_LN.append(correlation_ml_time_list_methods_LN)
    
    rmse_ml_space_list.append(rmse_ml_space_list_methods)
    rmse_ml_time_list.append(rmse_ml_time_list_methods)
    p_ml_space_list.append(p_ml_space_list_methods)
    p_ml_time_list.append(p_ml_time_list_methods)
    correlation_masked_ml_time_list_season_mean.append(correlation_masked_ml_time_list_method_mean)

    
    
correlation_ml_time_list_era20c_LN=correlation_ml_time_list_LN
correlation_ml_time_list_era20c_EN=correlation_ml_time_list_EN

correlation_afs_time_list_era20c_LN=correlation_afs_time_list_LN
correlation_afs_time_list_era20c_EN=correlation_afs_time_list_EN


correlation_ml_space_list_era20c=correlation_ml_space_list
correlation_ml_time_list_era20c=correlation_ml_time_list
rmse_ml_space_list_era20c=rmse_ml_space_list
rmse_ml_time_list_era20c=rmse_ml_time_list
p_ml_space_list_era20c=p_ml_space_list
p_ml_time_list_era20c=p_ml_time_list
correlation_masked_afs_time_list_season_max_era20c=correlation_masked_afs_time_list_season_max # len == season
correlation_masked_afs_time_list_season_mean_era20c=correlation_masked_afs_time_list_season_mean
correlation_masked_afs_time_list_season_min_era20c=correlation_masked_afs_time_list_season_min

correlation_afs_space_list_era20c=correlation_afs_space_list
correlation_afs_time_list_era20c=correlation_afs_time_list
rmse_afs_space_list_era20c=rmse_afs_space_list
rmse_afs_time_list_era20c=rmse_afs_time_list
p_afs_space_list_era20c=p_afs_space_list
p_afs_time_list_era20c=p_afs_time_list
correlation_masked_ml_time_list_season_mean_era20c=correlation_masked_ml_time_list_season_mean # len == season*method

In [ ]:
# era 20c no land
# this for 2m temperatures
correlation_afs_time_list_LN=[]
correlation_afs_time_list_EN=[]
correlation_ml_time_list_LN=[]
correlation_ml_time_list_EN=[]



correlation_afs_space_list=[]
correlation_afs_time_list=[]
rmse_afs_space_list=[]
rmse_afs_time_list=[]
p_afs_space_list=[]
p_afs_time_list=[]


correlation_ml_space_list=[]
correlation_ml_time_list=[]
rmse_ml_space_list=[]
rmse_ml_time_list=[]
p_ml_space_list=[]
p_ml_time_list=[]

naming_list=[]

correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]


for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]
    
    print(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")

    start_year=str(afs.forecast_time0.values[0])[0:4]
    print(start_year)
    
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    print(end_year)
    
    print(era20c_folder+"era20c_t2m_"+season+"_lowlow_anoms.nc")
    trcr=xr.open_dataset(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    trcr_time_long=trcr.time
    
    era20c=xr.open_dataset(era20c_folder+"era20c_t2m_"+season+"_lowlow_anoms.nc")
    era20c=era20c.sel(time=slice(start_year,end_year))
    era20c=era20c.where(test_mask_flipped)
    
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'2T_GDS0_SFC': 't2m'})
    afs["time"]=era20c.time
    afs=afs.where(test_mask_flipped)
    
    era20c_LN=era20c.sel(time=LN_nov_years_np,method="nearest")
    era20c_EN=era20c.sel(time=EN_nov_years_np,method="nearest")

    
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    

    
    r_afs_time = xs.pearson_r(afs["t2m"], era20c["t2m"], dim='time',skipna=True)
    
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["t2m"], era20c_LN["t2m"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["t2m"], era20c_EN["t2m"], dim='time')
        correlation_afs_time_list_LN.append(r_afs_time_LN)
        correlation_afs_time_list_EN.append(r_afs_time_EN)
    r_afs_space = xs.pearson_r(afs["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
    rmse_afs_time = xs.rmse(afs["t2m"], era20c["t2m"], dim='time',skipna=True)
    rmse_afs_space = xs.rmse(afs["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
    p_afs_time = xs.pearson_r_p_value(afs["t2m"], era20c["t2m"], dim="time",skipna=True)
    p_afs_space = xs.pearson_r_p_value(afs["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
    correlation_afs_space_list.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)

    correlation_masked_afs_time_list=[]
    #for member_index in np.arange(0,51):
    #    afs_single_member=afs.sel(member=member_index)
    #    outname="corr_afs_era20c_t2m_"+season+"_"+str(member_index)
    #    corr_file=corr_2d_sign(data1=afs_single_member,data2=era20c,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
    #    correlation_masked_afs_time_list.append(corr_file)
    #concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    #concat_mean=concat.mean(dim="member")
    #concat_min=concat.min(dim="member")
    #concat_max=concat.max(dim="member")
    #correlation_masked_afs_time_list_season_max.append(concat_max)
    #correlation_masked_afs_time_list_season_mean.append(concat_mean)
    #correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=trcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        ml=ml.where(test_mask_flipped)
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")



        #print(ml["t2m"].time.values)
        #print(era20c["t2m"].time.values)
        r_ml_time = xs.pearson_r(ml["t2m"], era20c["t2m"], dim='time',skipna=True)
        
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["t2m"], era20c_LN["t2m"], dim='time',skipna=True)
            r_ml_time_EN = xs.pearson_r(ml_EN["t2m"], era20c_EN["t2m"], dim='time',skipna=True)
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
    
        r_ml_space = xs.pearson_r(ml["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
        rmse_ml_time = xs.rmse(ml["t2m"], era20c["t2m"], dim='time',skipna=True)
        rmse_ml_space = xs.rmse(ml["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
        p_ml_time = xs.pearson_r_p_value(ml["t2m"], era20c["t2m"], dim="time",skipna=True)
        p_ml_space = xs.pearson_r_p_value(ml["t2m"], era20c["t2m"], dim=["lat", "lon"],skipna=True)
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)
        
        correlation_masked_ml_time_list=[]
        #for member_index in np.arange(0,5):
        #    outname="corr_"+method+"era20c_t2m_"+season+"_"+str(member_index)
        #    ml_single_member=ml.sel(member=member_index)
        #    corr_file=corr_2d_sign(data1=ml_single_member,data2=era20c,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        #    correlation_masked_ml_time_list.append(corr_file)
        #concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        #concat_mean=concat.mean(dim="member")
        
        #correlation_masked_ml_time_list_method_mean.append(concat_mean)
    

    
    if season=="DJF":
        correlation_ml_time_list_EN.append(correlation_ml_time_list_methods_EN)
        correlation_ml_time_list_LN.append(correlation_ml_time_list_methods_LN)
    correlation_ml_space_list.append(correlation_ml_space_list_methods)
    correlation_ml_time_list.append(correlation_ml_time_list_methods)
    rmse_ml_space_list.append(rmse_ml_space_list_methods)
    rmse_ml_time_list.append(rmse_ml_time_list_methods)
    p_ml_space_list.append(p_ml_space_list_methods)
    p_ml_time_list.append(p_ml_time_list_methods)
    correlation_masked_ml_time_list_season_mean.append(correlation_masked_ml_time_list_method_mean)


correlation_ml_space_list_era20c_landonly=correlation_ml_space_list
rmse_ml_space_list_era20c_landonly=rmse_ml_space_list
p_ml_space_list_era20c_landonly=p_ml_space_list

correlation_afs_space_list_era20c_landonly=correlation_afs_space_list
rmse_afs_space_list_era20c_landonly=rmse_afs_space_list
p_afs_space_list_era20c_landonly=p_afs_space_list

correlation_afs_time_list_era20c_landonly=correlation_afs_time_list
correlation_afs_time_list_era20c_landonly_LN=correlation_afs_time_list_LN
correlation_afs_time_list_era20c_landonly_EN=correlation_afs_time_list_EN
rmse_afs_time_list_era20c_landonly=rmse_afs_time_list
p_afs_time_list_era20c_landonly=p_afs_time_list

correlation_ml_time_list_era20c_landonly=correlation_ml_time_list
correlation_ml_time_list_era20c_landonly_LN=correlation_ml_time_list_LN
correlation_ml_time_list_era20c_landonly_EN=correlation_ml_time_list_EN
rmse_ml_time_list_era20c_landonly=rmse_ml_time_list
p_ml_time_list_era20c_landonly=p_ml_time_list


/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_CNN44_m*_anoms.nc
/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_unet1_small_m*_anoms.nc
/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_unet4_small_m*_anoms.nc


In [ ]:
# this for 2m temperatures, 20CR, landonly
correlation_afs_time_list_LN_landonly=[]
correlation_afs_time_list_EN_landonly=[]
correlation_ml_time_list_LN_landonly=[]
correlation_ml_time_list_EN_landonly=[]




correlation_afs_space_list_landonly=[]
correlation_afs_time_list=[]
rmse_afs_space_list_landonly=[]
rmse_afs_time_list=[]
p_afs_space_list_landonly=[]
p_afs_time_list=[]


correlation_ml_space_list_landonly=[]
correlation_ml_time_list=[]
rmse_ml_space_list_landonly=[]
rmse_ml_time_list=[]
p_ml_space_list_landonly=[]
p_ml_time_list=[]

naming_list=[]


correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]




for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]
    
    print(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")
    start_year=str(afs.forecast_time0.values[0])[0:4]
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    
    print(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr=xr.open_dataset(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr_time_long=tcr.time
    tcr=tcr.sel(time=slice(start_year,end_year))
    tcr=tcr.rename({'air': 't2m'})
    tcr=tcr.where(test_mask_flipped)
    
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'2T_GDS0_SFC': 't2m'})
    afs["time"]=tcr.time
    afs=afs.where(test_mask_flipped)
    
    tcr_LN=tcr.sel(time=LN_nov_years_np,method="nearest")
    tcr_EN=tcr.sel(time=EN_nov_years_np,method="nearest")

    
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    
    
    r_afs_time = xs.pearson_r(afs["t2m"], tcr["t2m"], dim='time',skipna=True)
    
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["t2m"], tcr_LN["t2m"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["t2m"], tcr_EN["t2m"], dim='time')
        correlation_afs_time_list_LN_landonly.append(r_afs_time_LN)
        correlation_afs_time_list_EN_landonly.append(r_afs_time_EN)
    r_afs_space = xs.pearson_r(afs["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
    rmse_afs_time = xs.rmse(afs["t2m"], tcr["t2m"], dim='time',skipna=True)
    rmse_afs_space = xs.rmse(afs["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
    p_afs_time = xs.pearson_r_p_value(afs["t2m"], tcr["t2m"], dim="time",skipna=True)
    p_afs_space = xs.pearson_r_p_value(afs["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
    correlation_afs_space_list_landonly.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list_landonly.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list_landonly.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)

    correlation_masked_afs_time_list=[]
    #for member_index in np.arange(0,51):
    #    afs_single_member=afs.sel(member=member_index)
    #    outname="corr_afs_tcr_t2m_"+season+"_"+str(member_index)
    #    corr_file=corr_2d_sign(data1=afs_single_member,data2=tcr,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
    #    correlation_masked_afs_time_list.append(corr_file)
    #concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    #concat_mean=concat.mean(dim="member")
    #concat_min=concat.min(dim="member")
    #concat_max=concat.max(dim="member")
    #correlation_masked_afs_time_list_season_max.append(concat_max)
    #correlation_masked_afs_time_list_season_mean.append(concat_mean)
    #correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=tcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        ml=ml.where(test_mask_flipped)
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")



        
        r_ml_time = xs.pearson_r(ml["t2m"], tcr["t2m"], dim='time',skipna=True)
        
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["t2m"], tcr_LN["t2m"], dim='time',skipna=True)
            r_ml_time_EN = xs.pearson_r(ml_EN["t2m"], tcr_EN["t2m"], dim='time',skipna=True)
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
            
        r_ml_space = xs.pearson_r(ml["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
        rmse_ml_time = xs.rmse(ml["t2m"], tcr["t2m"], dim='time',skipna=True)
        rmse_ml_space = xs.rmse(ml["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
        p_ml_time = xs.pearson_r_p_value(ml["t2m"], tcr["t2m"], dim="time",skipna=True)
        p_ml_space = xs.pearson_r_p_value(ml["t2m"], tcr["t2m"], dim=["lat", "lon"],skipna=True)
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)

        correlation_masked_ml_time_list=[]
        #for member_index in np.arange(0,5):
        #    outname="corr_"+method+"tcr_t2m_"+season+"_"+str(member_index)
        #    ml_single_member=ml.sel(member=member_index)
        #    corr_file=corr_2d_sign(data1=ml_single_member,data2=tcr,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        #    correlation_masked_ml_time_list.append(corr_file)
        #concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        #concat_mean=concat.mean(dim="member")
        
        correlation_masked_ml_time_list_method_mean.append(concat_mean)
    if season=="DJF":    
        correlation_ml_time_list_EN_landonly.append(correlation_ml_time_list_methods_EN)
        correlation_ml_time_list_LN_landonly.append(correlation_ml_time_list_methods_LN)
    correlation_ml_space_list_landonly.append(correlation_ml_space_list_methods)
    rmse_ml_space_list_landonly.append(rmse_ml_space_list_methods)
    p_ml_space_list_landonly.append(p_ml_space_list_methods)



/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_CNN44_m*_anoms.nc
/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_unet1_small_m*_anoms.nc
/Volumes/SPARK/CNN_seaspredc_analysis/ml_predicted_fields_postprocessed/best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_t2m_DJF_unet4_small_m*_anoms.nc


In [ ]:
# this for 2m temperatures, 20CR

correlation_afs_time_list_LN=[]
correlation_afs_time_list_EN=[]
correlation_ml_time_list_LN=[]
correlation_ml_time_list_EN=[]

correlation_afs_space_list=[]
correlation_afs_time_list=[]
rmse_afs_space_list=[]
rmse_afs_time_list=[]
p_afs_space_list=[]
p_afs_time_list=[]


correlation_ml_space_list=[]
correlation_ml_time_list=[]
rmse_ml_space_list=[]
rmse_ml_time_list=[]
p_ml_space_list=[]
p_ml_time_list=[]

naming_list=[]


correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]




for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]
    
    print(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"T2monthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")
    start_year=str(afs.forecast_time0.values[0])[0:4]
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    
    print(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr=xr.open_dataset(tcr_folder+"t2m_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr_time_long=tcr.time
    tcr=tcr.sel(time=slice(start_year,end_year))
    tcr=tcr.rename({'air': 't2m'})
    
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'2T_GDS0_SFC': 't2m'})
    afs["time"]=tcr.time
    
    tcr_LN=tcr.sel(time=LN_nov_years_np,method="nearest")
    tcr_EN=tcr.sel(time=EN_nov_years_np,method="nearest")

    
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    
    
    r_afs_time = xs.pearson_r(afs["t2m"], tcr["t2m"], dim='time')
    
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["t2m"], tcr_LN["t2m"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["t2m"], tcr_EN["t2m"], dim='time')
        correlation_afs_time_list_LN.append(r_afs_time_LN)
        correlation_afs_time_list_EN.append(r_afs_time_EN)
    r_afs_space = xs.pearson_r(afs["t2m"], tcr["t2m"], dim=["lat", "lon"])
    rmse_afs_time = xs.rmse(afs["t2m"], tcr["t2m"], dim='time')
    rmse_afs_space = xs.rmse(afs["t2m"], tcr["t2m"], dim=["lat", "lon"])
    p_afs_time = xs.pearson_r_p_value(afs["t2m"], tcr["t2m"], dim="time")
    p_afs_space = xs.pearson_r_p_value(afs["t2m"], tcr["t2m"], dim=["lat", "lon"])
    correlation_afs_space_list.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)

    correlation_masked_afs_time_list=[]
    for member_index in np.arange(0,51):
        afs_single_member=afs.sel(member=member_index)
        outname="corr_afs_tcr_t2m_"+season+"_"+str(member_index)
        corr_file=corr_2d_sign(data1=afs_single_member,data2=tcr,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        correlation_masked_afs_time_list.append(corr_file)
    concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    concat_mean=concat.mean(dim="member")
    concat_min=concat.min(dim="member")
    concat_max=concat.max(dim="member")
    correlation_masked_afs_time_list_season_max.append(concat_max)
    correlation_masked_afs_time_list_season_mean.append(concat_mean)
    correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_t2m_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=tcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")



        
        r_ml_time = xs.pearson_r(ml["t2m"], tcr["t2m"], dim='time')
        
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["t2m"], tcr_LN["t2m"], dim='time')
            r_ml_time_EN = xs.pearson_r(ml_EN["t2m"], tcr_EN["t2m"], dim='time')
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
        
        r_ml_space = xs.pearson_r(ml["t2m"], tcr["t2m"], dim=["lat", "lon"])
        rmse_ml_time = xs.rmse(ml["t2m"], tcr["t2m"], dim='time')
        rmse_ml_space = xs.rmse(ml["t2m"], tcr["t2m"], dim=["lat", "lon"])
        p_ml_time = xs.pearson_r_p_value(ml["t2m"], tcr["t2m"], dim="time")
        p_ml_space = xs.pearson_r_p_value(ml["t2m"], tcr["t2m"], dim=["lat", "lon"])
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)

        correlation_masked_ml_time_list=[]
        for member_index in np.arange(0,5):
            outname="corr_"+method+"tcr_t2m_"+season+"_"+str(member_index)
            ml_single_member=ml.sel(member=member_index)
            corr_file=corr_2d_sign(data1=ml_single_member,data2=tcr,var1="t2m",var2="t2m",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
            correlation_masked_ml_time_list.append(corr_file)
        concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        concat_mean=concat.mean(dim="member")
        
        correlation_masked_ml_time_list_method_mean.append(concat_mean)
        
    if season=="DJF":
        correlation_ml_time_list_LN.append(correlation_ml_time_list_methods_LN)
        correlation_ml_time_list_EN.append(correlation_ml_time_list_methods_EN)
    correlation_ml_space_list.append(correlation_ml_space_list_methods)
    correlation_ml_time_list.append(correlation_ml_time_list_methods)
    rmse_ml_space_list.append(rmse_ml_space_list_methods)
    rmse_ml_time_list.append(rmse_ml_time_list_methods)
    p_ml_space_list.append(p_ml_space_list_methods)
    p_ml_time_list.append(p_ml_time_list_methods)
    correlation_masked_ml_time_list_season_mean.append(correlation_masked_ml_time_list_method_mean)

# Fig 7 and S13

In [ ]:
ml_time_LN_landonly_tcr=xr.concat([correlation_ml_time_list_LN_landonly[0][0],correlation_ml_time_list_LN_landonly[0][1],correlation_ml_time_list_LN_landonly[0][2],correlation_ml_time_list_LN_landonly[0][3],correlation_ml_time_list_LN_landonly[0][4],correlation_ml_time_list_LN_landonly[0][5]],dim="height")
ml_time_LN_landonly_era20c=xr.concat([correlation_ml_time_list_era20c_landonly_LN[0][0],correlation_ml_time_list_era20c_landonly_LN[0][1],correlation_ml_time_list_era20c_landonly_LN[0][2],correlation_ml_time_list_era20c_landonly_LN[0][3],correlation_ml_time_list_era20c_landonly_LN[0][4],correlation_ml_time_list_era20c_landonly_LN[0][5]],dim="height")
ml_time_EN_landonly_tcr=xr.concat([correlation_ml_time_list_EN_landonly[0][0],correlation_ml_time_list_EN_landonly[0][1],correlation_ml_time_list_EN_landonly[0][2],correlation_ml_time_list_EN_landonly[0][3],correlation_ml_time_list_EN_landonly[0][4],correlation_ml_time_list_EN_landonly[0][5]],dim="height")
ml_time_EN_landonly_era20c=xr.concat([correlation_ml_time_list_era20c_landonly_EN[0][0],correlation_ml_time_list_era20c_landonly_EN[0][1],correlation_ml_time_list_era20c_landonly_EN[0][2],correlation_ml_time_list_era20c_landonly_EN[0][3],correlation_ml_time_list_era20c_landonly_EN[0][4],correlation_ml_time_list_era20c_landonly_EN[0][5]],dim="height")

ds_merged_ens_tcr=xr.concat([ml_time_EN_landonly_tcr.mean(dim="height").mean(dim="member"),ml_time_LN_landonly_tcr.mean(dim="height").mean(dim="member"),ml_time_EN_landonly_tcr.mean(dim="height").mean(dim="member")-ml_time_LN_landonly_tcr.mean(dim="height").mean(dim="member"),correlation_afs_time_list_EN_landonly[0].mean(dim="member"),correlation_afs_time_list_LN_landonly[0].mean(dim="member"),correlation_afs_time_list_EN_landonly[0].mean(dim="member")-correlation_afs_time_list_LN_landonly[0].mean(dim="member")],dim="height")
ds_merged_ens_era20c=xr.concat([ml_time_EN_landonly_era20c.mean(dim="height").mean(dim="member"),ml_time_LN_landonly_era20c.mean(dim="height").mean(dim="member"),ml_time_EN_landonly_era20c.mean(dim="height").mean(dim="member")-ml_time_LN_landonly_era20c.mean(dim="height").mean(dim="member"),correlation_afs_time_list_era20c_landonly_EN[0].mean(dim="member"),correlation_afs_time_list_era20c_landonly_LN[0].mean(dim="member"),correlation_afs_time_list_era20c_landonly_EN[0].mean(dim="member")-correlation_afs_time_list_era20c_landonly_LN[0].mean(dim="member")],dim="height")

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(14, 5), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
axs=axs.flatten()

#Loop over all of the models
for i in range(len(INDEX)):
    tplot=ds_merged_ens_tcr[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(ds_merged_ens_tcr[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    #axs[i].set_title(methods_4_plots_publish[i], fontsize=5)

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.05, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_corr_ENSO_tcr.pdf")
fig.savefig(output_folder+"t2m_corr_ENSO_tcr.png")




In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(14, 5), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
INDEX=list(["(a)","(b)","(c)","(d)","(e)","(f)"])
axs=axs.flatten()

#Loop over all of the models
for i in range(len(INDEX)):
    tplot=ds_merged_ens_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(ds_merged_ens_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    #axs[i].set_title(methods_4_plots_publish[i], fontsize=5)

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.05, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_corr_ENSO_era20c.pdf")
fig.savefig(output_folder+"t2m_corr_ENSO_era20c.png")



# Fig 1 and S2

In [ ]:
seasons=["jun","jul","aug","JJA"]

empty_list_ensmax=[]
empty_list_ensmean=[]
empty_list_ensmin=[]
empty_list_reg=[]
empty_list_CNN=[]
empty_list_UNET=[]

# 0:4 is summer, 4:8 is winter

for p in np.arange(0,4):
    ds_unets=xr.concat([correlation_ml_time_list[p][4].mean(axis=0),correlation_ml_time_list[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([correlation_ml_time_list[p][1].mean(axis=0),correlation_ml_time_list[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_ensmax.append(correlation_afs_time_list[p].max(axis=0))
    empty_list_ensmean.append(correlation_afs_time_list[p].mean(axis=0))
    empty_list_ensmin.append(correlation_afs_time_list[p].min(axis=0))
    empty_list_reg.append(correlation_ml_time_list[p][0].mean(axis=0))
    empty_list_CNN.append(ds_cnns_average)
    empty_list_UNET.append(ds_unets_average)
    
ensmaxes_summer_correlation=xr.concat(empty_list_ensmax,dim="season")
ensmeans_summer_correlation=xr.concat(empty_list_ensmean,dim="season")
ensmins_summer_correlation=xr.concat(empty_list_ensmin,dim="season")
reg_summer_correlation=xr.concat(empty_list_reg,dim="season")
CNN_summer_correlation=xr.concat(empty_list_CNN,dim="season")
UNET_summer_correlation=xr.concat(empty_list_UNET,dim="season")


all_methods_summer_correlation=xr.concat([ensmaxes_summer_correlation,ensmeans_summer_correlation,ensmins_summer_correlation,reg_summer_correlation,CNN_summer_correlation,UNET_summer_correlation],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_correlation[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_correlation[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allsummer.pdf")
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allsummer.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_summer_correlation=all_methods_summer_correlation.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_correlation[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_correlation[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allsummer_onlyland.pdf")
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allsummer_onlyland.png")




In [ ]:


empty_list_era20c_ensmax=[]
empty_list_era20c_ensmean=[]
empty_list_era20c_ensmin=[]
empty_list_era20c_reg=[]
empty_list_era20c_CNN=[]
empty_list_era20c_UNET=[]

# 0:4 is summer, 4:8 is winter

for p in np.arange(0,4):
    ds_unets=xr.concat([correlation_ml_time_list_era20c[p][4].mean(axis=0),correlation_ml_time_list_era20c[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([correlation_ml_time_list_era20c[p][1].mean(axis=0),correlation_ml_time_list_era20c[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_era20c_ensmax.append(correlation_afs_time_list_era20c[p].max(axis=0))
    empty_list_era20c_ensmean.append(correlation_afs_time_list_era20c[p].mean(axis=0))
    empty_list_era20c_ensmin.append(correlation_afs_time_list_era20c[p].min(axis=0))
    empty_list_era20c_reg.append(correlation_ml_time_list_era20c[p][0].mean(axis=0))
    empty_list_era20c_CNN.append(ds_cnns_average)
    empty_list_era20c_UNET.append(ds_unets_average)
    
ensmaxes_summer_correlation_era20c=xr.concat(empty_list_era20c_ensmax,dim="season")
ensmeans_summer_correlation_era20c=xr.concat(empty_list_era20c_ensmean,dim="season")
ensmins_summer_correlation_era20c=xr.concat(empty_list_era20c_ensmin,dim="season")
reg_summer_correlation_era20c=xr.concat(empty_list_era20c_reg,dim="season")
CNN_summer_correlation_era20c=xr.concat(empty_list_era20c_CNN,dim="season")
UNET_summer_correlation_era20c=xr.concat(empty_list_era20c_UNET,dim="season")


all_methods_summer_correlation_era20c_era20c=xr.concat([ensmaxes_summer_correlation_era20c,ensmeans_summer_correlation_era20c,ensmins_summer_correlation_era20c,reg_summer_correlation_era20c,CNN_summer_correlation_era20c,UNET_summer_correlation_era20c],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_correlation_era20c_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_correlation_era20c_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allsummer.pdf")
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allsummer.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_summer_correlation_era20c_era20c=all_methods_summer_correlation_era20c_era20c.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_correlation_era20c_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_correlation_era20c_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allsummer_onlyland.pdf")
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allsummer_onlyland.png")





# Fig 3 and S8

In [ ]:
empty_list_ensmax=[]
empty_list_ensmean=[]
empty_list_ensmin=[]
empty_list_reg=[]
empty_list_CNN=[]
empty_list_UNET=[]

# 0:4 is summer, 4:8 is winter

for p in np.arange(4,8):
    ds_unets=xr.concat([correlation_ml_time_list[p][4].mean(axis=0),correlation_ml_time_list[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([correlation_ml_time_list[p][1].mean(axis=0),correlation_ml_time_list[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_ensmax.append(correlation_afs_time_list[p].max(axis=0))
    empty_list_ensmean.append(correlation_afs_time_list[p].mean(axis=0))
    empty_list_ensmin.append(correlation_afs_time_list[p].min(axis=0))
    empty_list_reg.append(correlation_ml_time_list[p][0].mean(axis=0))
    empty_list_CNN.append(ds_cnns_average)
    empty_list_UNET.append(ds_unets_average)
    
ensmaxes_winter_correlation=xr.concat(empty_list_ensmax,dim="season")
ensmeans_winter_correlation=xr.concat(empty_list_ensmean,dim="season")
ensmins_winter_correlation=xr.concat(empty_list_ensmin,dim="season")
reg_winter_correlation=xr.concat(empty_list_reg,dim="season")
CNN_winter_correlation=xr.concat(empty_list_CNN,dim="season")
UNET_winter_correlation=xr.concat(empty_list_UNET,dim="season")


all_methods_winter_correlation=xr.concat([ensmaxes_winter_correlation,ensmeans_winter_correlation,ensmins_winter_correlation,reg_winter_correlation,CNN_winter_correlation,UNET_winter_correlation],dim="season")




INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_correlation[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_correlation[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allwinter.pdf")
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allwinter.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_winter_correlation=all_methods_winter_correlation.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_correlation[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_correlation[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allwinter_onlyland.pdf")
fig.savefig(output_folder+"t2m_correlation_afs_time_publish_allwinter_onlyland.png")





In [ ]:

empty_list_era20c_ensmax=[]
empty_list_era20c_ensmean=[]
empty_list_era20c_ensmin=[]
empty_list_era20c_reg=[]
empty_list_era20c_CNN=[]
empty_list_era20c_UNET=[]

# 0:4 is winter, 4:8 is winter

for p in np.arange(4,8):
    ds_unets=xr.concat([correlation_ml_time_list_era20c[p][4].mean(axis=0),correlation_ml_time_list_era20c[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([correlation_ml_time_list_era20c[p][1].mean(axis=0),correlation_ml_time_list_era20c[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_era20c_ensmax.append(correlation_afs_time_list_era20c[p].max(axis=0))
    empty_list_era20c_ensmean.append(correlation_afs_time_list_era20c[p].mean(axis=0))
    empty_list_era20c_ensmin.append(correlation_afs_time_list_era20c[p].min(axis=0))
    empty_list_era20c_reg.append(correlation_ml_time_list_era20c[p][0].mean(axis=0))
    empty_list_era20c_CNN.append(ds_cnns_average)
    empty_list_era20c_UNET.append(ds_unets_average)
    
ensmaxes_winter_correlation_era20c=xr.concat(empty_list_era20c_ensmax,dim="season")
ensmeans_winter_correlation_era20c=xr.concat(empty_list_era20c_ensmean,dim="season")
ensmins_winter_correlation_era20c=xr.concat(empty_list_era20c_ensmin,dim="season")
reg_winter_correlation_era20c=xr.concat(empty_list_era20c_reg,dim="season")
CNN_winter_correlation_era20c=xr.concat(empty_list_era20c_CNN,dim="season")
UNET_winter_correlation_era20c=xr.concat(empty_list_era20c_UNET,dim="season")


all_methods_winter_correlation_era20c_era20c=xr.concat([ensmaxes_winter_correlation_era20c,ensmeans_winter_correlation_era20c,ensmins_winter_correlation_era20c,reg_winter_correlation_era20c,CNN_winter_correlation_era20c,UNET_winter_correlation_era20c],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_correlation_era20c_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_correlation_era20c_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allwinter.pdf")
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allwinter.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_winter_correlation_era20c_era20c=all_methods_winter_correlation_era20c_era20c.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_correlation_era20c_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_correlation_era20c_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allwinter_onlyland.pdf")
fig.savefig(output_folder+"t2m_era20c_correlation_afs_time_publish_allwinter_onlyland.png")






# Fig S1 and S3

In [ ]:
seasons=["jun","jul","aug","JJA"]

empty_list_ensmax=[]
empty_list_ensmean=[]
empty_list_ensmin=[]
empty_list_reg=[]
empty_list_CNN=[]
empty_list_UNET=[]

# 0:4 is summer, 4:8 is winter

for p in np.arange(0,4):
    ds_unets=xr.concat([rmse_ml_time_list[p][4].mean(axis=0),rmse_ml_time_list[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([rmse_ml_time_list[p][1].mean(axis=0),rmse_ml_time_list[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_ensmax.append(rmse_afs_time_list[p].max(axis=0))
    empty_list_ensmean.append(rmse_afs_time_list[p].mean(axis=0))
    empty_list_ensmin.append(rmse_afs_time_list[p].min(axis=0))
    empty_list_reg.append(rmse_ml_time_list[p][0].mean(axis=0))
    empty_list_CNN.append(ds_cnns_average)
    empty_list_UNET.append(ds_unets_average)
    
ensmaxes_summer_rmse=xr.concat(empty_list_ensmax,dim="season")
ensmeans_summer_rmse=xr.concat(empty_list_ensmean,dim="season")
ensmins_summer_rmse=xr.concat(empty_list_ensmin,dim="season")
reg_summer_rmse=xr.concat(empty_list_reg,dim="season")
CNN_summer_rmse=xr.concat(empty_list_CNN,dim="season")
UNET_summer_rmse=xr.concat(empty_list_UNET,dim="season")


all_methods_summer_rmse=xr.concat([ensmaxes_summer_rmse,ensmeans_summer_rmse,ensmins_summer_rmse,reg_summer_rmse,CNN_summer_rmse,UNET_summer_rmse],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_rmse[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_rmse[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allsummer.pdf")
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allsummer.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_summer_rmse=all_methods_summer_rmse.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_rmse[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_rmse[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allsummer_onlyland.pdf")
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allsummer_onlyland.png")





In [ ]:
seasons=["jun","jul","aug","JJA"]

empty_list_era20c_ensmax=[]
empty_list_era20c_ensmean=[]
empty_list_era20c_ensmin=[]
empty_list_era20c_reg=[]
empty_list_era20c_CNN=[]
empty_list_era20c_UNET=[]

# 0:4 is summer, 4:8 is winter

for p in np.arange(0,4):
    ds_unets=xr.concat([rmse_ml_time_list_era20c[p][4].mean(axis=0),rmse_ml_time_list_era20c[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([rmse_ml_time_list_era20c[p][1].mean(axis=0),rmse_ml_time_list_era20c[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_era20c_ensmax.append(rmse_afs_time_list_era20c[p].max(axis=0))
    empty_list_era20c_ensmean.append(rmse_afs_time_list_era20c[p].mean(axis=0))
    empty_list_era20c_ensmin.append(rmse_afs_time_list_era20c[p].min(axis=0))
    empty_list_era20c_reg.append(rmse_ml_time_list_era20c[p][0].mean(axis=0))
    empty_list_era20c_CNN.append(ds_cnns_average)
    empty_list_era20c_UNET.append(ds_unets_average)
    
ensmaxes_summer_rmse_era20c=xr.concat(empty_list_era20c_ensmax,dim="season")
ensmeans_summer_rmse_era20c=xr.concat(empty_list_era20c_ensmean,dim="season")
ensmins_summer_rmse_era20c=xr.concat(empty_list_era20c_ensmin,dim="season")
reg_summer_rmse_era20c=xr.concat(empty_list_era20c_reg,dim="season")
CNN_summer_rmse_era20c=xr.concat(empty_list_era20c_CNN,dim="season")
UNET_summer_rmse_era20c=xr.concat(empty_list_era20c_UNET,dim="season")


all_methods_summer_rmse_era20c=xr.concat([ensmaxes_summer_rmse_era20c,ensmeans_summer_rmse_era20c,ensmins_summer_rmse_era20c,reg_summer_rmse_era20c,CNN_summer_rmse_era20c,UNET_summer_rmse_era20c],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_rmse_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_rmse_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allsummer.pdf")
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allsummer.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_summer_rmse_era20c=all_methods_summer_rmse_era20c.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_summer_rmse_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_summer_rmse_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allsummer_onlyland.pdf")
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allsummer_onlyland.png")






# Fig S7 and S9

In [ ]:
seasons=["jun","jul","aug","JJA"]

empty_list_ensmax=[]
empty_list_ensmean=[]
empty_list_ensmin=[]
empty_list_reg=[]
empty_list_CNN=[]
empty_list_UNET=[]

# 0:4 is winter, 4:8 is winter

for p in np.arange(4,8):
    ds_unets=xr.concat([rmse_ml_time_list[p][4].mean(axis=0),rmse_ml_time_list[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([rmse_ml_time_list[p][1].mean(axis=0),rmse_ml_time_list[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_ensmax.append(rmse_afs_time_list[p].max(axis=0))
    empty_list_ensmean.append(rmse_afs_time_list[p].mean(axis=0))
    empty_list_ensmin.append(rmse_afs_time_list[p].min(axis=0))
    empty_list_reg.append(rmse_ml_time_list[p][0].mean(axis=0))
    empty_list_CNN.append(ds_cnns_average)
    empty_list_UNET.append(ds_unets_average)
    
ensmaxes_winter_rmse=xr.concat(empty_list_ensmax,dim="season")
ensmeans_winter_rmse=xr.concat(empty_list_ensmean,dim="season")
ensmins_winter_rmse=xr.concat(empty_list_ensmin,dim="season")
reg_winter_rmse=xr.concat(empty_list_reg,dim="season")
CNN_winter_rmse=xr.concat(empty_list_CNN,dim="season")
UNET_winter_rmse=xr.concat(empty_list_UNET,dim="season")


all_methods_winter_rmse=xr.concat([ensmaxes_winter_rmse,ensmeans_winter_rmse,ensmins_winter_rmse,reg_winter_rmse,CNN_winter_rmse,UNET_winter_rmse],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_rmse[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_rmse[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allwinter.pdf")
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allwinter.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_winter_rmse=all_methods_winter_rmse.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_rmse[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_rmse[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allwinter_onlyland.pdf")
fig.savefig(output_folder+"t2m_rmse_afs_time_publish_allwinter_onlyland.png")






In [ ]:
seasons=["jun","jul","aug","JJA"]

empty_list_era20c_ensmax=[]
empty_list_era20c_ensmean=[]
empty_list_era20c_ensmin=[]
empty_list_era20c_reg=[]
empty_list_era20c_CNN=[]
empty_list_era20c_UNET=[]

# 0:4 is winter, 4:8 is winter

for p in np.arange(4,8):
    ds_unets=xr.concat([rmse_ml_time_list_era20c[p][4].mean(axis=0),rmse_ml_time_list_era20c[p][5].mean(axis=0)],dim="height")
    ds_cnns=xr.concat([rmse_ml_time_list_era20c[p][1].mean(axis=0),rmse_ml_time_list_era20c[p][2].mean(axis=0)],dim="height")
    ds_cnns_average=ds_cnns.mean(dim="height")
    ds_unets_average=ds_unets.mean(dim="height")
    empty_list_era20c_ensmax.append(rmse_afs_time_list_era20c[p].max(axis=0))
    empty_list_era20c_ensmean.append(rmse_afs_time_list_era20c[p].mean(axis=0))
    empty_list_era20c_ensmin.append(rmse_afs_time_list_era20c[p].min(axis=0))
    empty_list_era20c_reg.append(rmse_ml_time_list_era20c[p][0].mean(axis=0))
    empty_list_era20c_CNN.append(ds_cnns_average)
    empty_list_era20c_UNET.append(ds_unets_average)
    
ensmaxes_winter_rmse_era20c=xr.concat(empty_list_era20c_ensmax,dim="season")
ensmeans_winter_rmse_era20c=xr.concat(empty_list_era20c_ensmean,dim="season")
ensmins_winter_rmse_era20c=xr.concat(empty_list_era20c_ensmin,dim="season")
reg_winter_rmse_era20c=xr.concat(empty_list_era20c_reg,dim="season")
CNN_winter_rmse_era20c=xr.concat(empty_list_era20c_CNN,dim="season")
UNET_winter_rmse_era20c=xr.concat(empty_list_era20c_UNET,dim="season")


all_methods_winter_rmse_era20c=xr.concat([ensmaxes_winter_rmse_era20c,ensmeans_winter_rmse_era20c,ensmins_winter_rmse_era20c,reg_winter_rmse_era20c,CNN_winter_rmse_era20c,UNET_winter_rmse_era20c],dim="season")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_rmse_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_rmse_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allwinter.pdf")
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allwinter.png")

INDEX=list(["(a)","(g)","(m)","(s)","(b)","(h)","(n)","(t)","(c)","(i)","(o)","(u)","(d)","(j)","(p)","(v)","(e)","(k)","(q)","(w)","(f)","(l)","(r)","(x)"])

fig, axs = plt.subplots(6, 4, figsize=(12, 10), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()
all_methods_winter_rmse_era20c=all_methods_winter_rmse_era20c.where(test_mask_flipped)


#Loop over all of the models
for i in range(24):
    tplot=all_methods_winter_rmse_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, transform=ccrs.PlateCarree(), cmap="viridis",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(all_methods_winter_rmse_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    axs[i].set_title("")

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.01, hspace=0.005)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('RMSE [K]', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allwinter_onlyland.pdf")
fig.savefig(output_folder+"t2m_era20c_rmse_afs_time_publish_allwinter_onlyland.png")







# Fig S5 / S11

In [ ]:
for p in range(len(correlation_ml_space_list_era20c)):
    methods_4_plots=["ENS MAX ASF","ENS AVE ASF","ENS MIN ASF",'reg_', 'CNN8_', 'CNN11kiri_', 'CNN44_', 'unet1_small_', 'unet4_small_']
    seasons=["jun","jul","aug","JJA","dec","jan","feb","DJF"]
    
    season=seasons[p]



    correlation_ml_space_allmethods_era20c=xr.concat(correlation_ml_space_list_era20c[p],dim="methods")


    fig = plt.figure(figsize=(10, 5), dpi= 300)
    for i in range(51):
        correlation_afs_space_list_era20c[p][i].plot(color="k",linestyle="dotted",alpha=0.3)
    correlation_afs_space_list_era20c[p].mean(dim="member").plot(color="k",alpha=1,label="AFS20C")
    #r_space_list[50].plot(color="k",linestyle="--",alpha=1,label="ASF20C member")
    # we kick out CNN8
    for b in [0,2,3,4,5]:
        print(methods_paper[b])
        print(correlation_ml_space_list_era20c[p][b].mean(dim="member").mean(dim="time").values)
        for a in range(5):
            correlation_ml_space_list_era20c[p][b][a].plot(color="red",linestyle="dotted",alpha=0.3)
    correlation_ml_space_allmethods_era20c.mean(dim="methods").mean(dim="member").plot(color="red",alpha=1,label="ML ensemble")

    corr_of_corr=xr.corr(correlation_ml_space_allmethods_era20c.mean(dim="methods").mean(dim="member"),correlation_afs_space_list_era20c[p].mean(dim="member"),dim="time")
    
    plt.legend()
    plt.title(season+" t2m"+" R Space "+str(np.round(corr_of_corr.values,2)))
    plt.xlabel("Time")
    plt.ylabel("R")
    plt.ylim(-1,1)
    plt.grid()
    plt.show()
    fig.savefig(output_folder+season+"_t2m_era20c_correlation_afs_space_v2.pdf")
    fig.savefig(output_folder+season+"_t2m_era20c_correlation_afs_space_v2.png")

# Fig S6 / S12

In [ ]:
for p in range(len(rmse_ml_space_list_era20c)):
    methods_4_plots=["ENS MAX ASF","ENS AVE ASF","ENS MIN ASF",'reg_', 'CNN8_', 'CNN11kiri_', 'CNN44_', 'unet1_small_', 'unet4_small_']
    seasons=["jun","jul","aug","JJA","dec","jan","feb","DJF"]
    
    season=seasons[p]



    rmse_ml_space_allmethods_era20c=xr.concat(rmse_ml_space_list_era20c[p],dim="methods")


    fig = plt.figure(figsize=(10, 5), dpi= 300)
    for i in range(51):
        rmse_afs_space_list_era20c[p][i].plot(color="k",linestyle="dotted",alpha=0.3)
    
    
    #r_space_list[50].plot(color="k",linestyle="--",alpha=1,label="ASF20C member")
    # we kick out CNN8
    for b in [0,2,3,4,5]:
        print(methods_paper[b])
        print(rmse_ml_space_list_era20c[p][b].mean(dim="member").mean(dim="time").values)
        for a in range(5):
            rmse_ml_space_list_era20c[p][b][a].plot(color="red",linestyle="dotted",alpha=0.3)
    rmse_ml_space_allmethods_era20c.mean(dim="methods").mean(dim="member").plot(color="red",alpha=1,label="ML ensemble")

    corr_of_corr=xr.corr(rmse_ml_space_allmethods_era20c.mean(dim="methods").mean(dim="member"),rmse_afs_space_list_era20c[p].mean(dim="member"),dim="time")
    
    plt.legend()
    plt.title(season+" t2m"+" RMSE Space "+str(np.round(corr_of_corr.values,2)))
    plt.xlabel("Time")
    plt.ylabel("RMSE [K]")
    plt.ylim()
    plt.grid()
    plt.show()
    fig.savefig(output_folder+season+"_t2m_era20c_rmse_afs_space_v2.pdf")
    fig.savefig(output_folder+season+"_t2m_era20c_rmse_afs_space_v2.png")

# Fig 2 / 4

In [ ]:
for p in range(len(correlation_ml_space_list)):
    methods_4_plots=["ENS MAX ASF","ENS AVE ASF","ENS MIN ASF",'reg_', 'CNN8_', 'CNN11kiri_', 'CNN44_', 'unet1_small_', 'unet4_small_']
    seasons=["jun","jul","aug","JJA","dec","jan","feb","DJF"]
    
    season=seasons[p]



    correlation_ml_space_allmethods=xr.concat(correlation_ml_space_list[p],dim="methods")


    fig = plt.figure(figsize=(10, 5), dpi= 300)
    for i in range(51):
        correlation_afs_space_list[p][i].plot(color="k",linestyle="dotted",alpha=0.3)
    correlation_afs_space_list[p].mean(dim="member").plot(color="k",alpha=1,label="AFS20C")
    #r_space_list[50].plot(color="k",linestyle="--",alpha=1,label="ASF20C member")
    # we kick out CNN8
    for b in [0,2,3,4,5]:
    #for b in range(6):
        print(methods_paper[b])
        print(correlation_ml_space_list[p][b].mean(dim="member").mean(dim="time").values)
        for a in range(5):
            correlation_ml_space_list[p][b][a].plot(color="red",linestyle="dotted",alpha=0.3)
    correlation_ml_space_allmethods.mean(dim="methods").mean(dim="member").plot(color="red",alpha=1,label="ML ensemble")

    corr_of_corr=xr.corr(correlation_ml_space_allmethods.mean(dim="methods").mean(dim="member"),correlation_afs_space_list[p].mean(dim="member"),dim="time")
    
    plt.legend()
    plt.title(season+" t2m"+" R Space "+str(np.round(corr_of_corr.values,2)))
    plt.xlabel("Time")
    plt.ylabel("R")
    plt.ylim(-1,1)
    plt.grid()
    plt.show()
    fig.savefig(output_folder+season+"_t2m_correlation_afs_space_v2.pdf")
    fig.savefig(output_folder+season+"_t2m_correlation_afs_space_v2.png")

# Fig S4 / S10

In [ ]:
for p in range(len(rmse_ml_space_list)):
    methods_4_plots=["ENS MAX ASF","ENS AVE ASF","ENS MIN ASF",'reg_', 'CNN8_', 'CNN11kiri_', 'CNN44_', 'unet1_small_', 'unet4_small_']
    seasons=["jun","jul","aug","JJA","dec","jan","feb","DJF"]
    
    season=seasons[p]



    rmse_ml_space_allmethods=xr.concat(rmse_ml_space_list[p],dim="methods")


    fig = plt.figure(figsize=(10, 5), dpi= 300)
    for i in range(51):
        rmse_afs_space_list[p][i].plot(color="k",linestyle="dotted",alpha=0.3)
    rmse_afs_space_list[p].mean(dim="member").plot(color="k",alpha=1,label="AFS20C")
    #r_space_list[50].plot(color="k",linestyle="--",alpha=1,label="ASF20C member")
    # we kick out CNN8
    for b in [0,2,3,4,5]:
        print(methods_paper[b])
        print(rmse_ml_space_list[p][b].mean(dim="member").mean(dim="time").values)
        for a in range(5):
            rmse_ml_space_list[p][b][a].plot(color="red",linestyle="dotted",alpha=0.3)
    rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member").plot(color="red",alpha=1,label="ML ensemble")

    corr_of_corr=xr.corr(rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member"),rmse_afs_space_list[p].mean(dim="member"),dim="time")
    
    plt.legend()
    plt.title(season+" t2m"+" RMSE Space "+str(np.round(corr_of_corr.values,2)))
    plt.xlabel("Time")
    plt.ylabel("RMSE [K]")
    plt.ylim()
    plt.grid()
    plt.show()
    fig.savefig(output_folder+season+"_t2m_rmse_afs_space_v2.pdf")
    fig.savefig(output_folder+season+"_t2m_rmse_afs_space_v2.png")

In [ ]:
mask_upper=rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member")>rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member").quantile(0.75).values
mask_lower=rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member")<rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member").quantile(0.25).values
ml_initial_condition_years_RMSEhigh_DJF_t2m=rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member").where(mask_upper==1,drop=True).time.values
ml_initial_condition_years_RMSElow_DJF_t2m=rmse_ml_space_allmethods.mean(dim="methods").mean(dim="member").where(mask_lower==1,drop=True).time.values
#
mask_upper=rmse_afs_space_list[p].mean(dim="member")>rmse_afs_space_list[p].mean(dim="member").quantile(0.75).values
mask_lower=rmse_afs_space_list[p].mean(dim="member")<rmse_afs_space_list[p].mean(dim="member").quantile(0.25).values
afs_initial_condition_years_RMSEhigh_DJF_t2m=rmse_afs_space_list[p].mean(dim="member").where(mask_upper==1,drop=True).time.values
afs_initial_condition_years_RMSElow_DJF_t2m=rmse_afs_space_list[p].mean(dim="member").where(mask_lower==1,drop=True).time.values

In [ ]:
####

# processing SLP data

In [ ]:
# era 20c
# this for slp
seasons=["JJA","DJF"]

correlation_afs_time_list_LN=[]
correlation_afs_time_list_EN=[]
correlation_ml_time_list_LN=[]
correlation_ml_time_list_EN=[]


correlation_afs_space_list=[]
correlation_afs_time_list=[]
rmse_afs_space_list=[]
rmse_afs_time_list=[]
p_afs_space_list=[]
p_afs_time_list=[]


correlation_ml_space_list=[]
correlation_ml_time_list=[]
rmse_ml_space_list=[]
rmse_ml_time_list=[]
p_ml_space_list=[]
p_ml_time_list=[]

naming_list=[]

correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]




for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]

    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]

    
    print(asf20c_folder+"SLPmonthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"SLPmonthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")

    start_year=str(afs.forecast_time0.values[0])[0:4]
    print(start_year)
    
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    print(end_year)
    
    print(era20c_folder+"era20c_msl_"+season+"_lowlow_anoms.nc")
    tcr=xr.open_dataset(tcr_folder+"slp_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr_time_long=tcr.time
    
    era20c=xr.open_dataset(era20c_folder+"era20c_msl_"+season+"_lowlow_anoms.nc")
    era20c=era20c.sel(time=slice(start_year,end_year))
    
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'MSL_GDS0_SFC': 'msl'})
    afs["time"]=era20c.time
    
    era20c_LN=era20c.sel(time=LN_nov_years_np,method="nearest")
    era20c_EN=era20c.sel(time=EN_nov_years_np,method="nearest")
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    
    
    r_afs_time = xs.pearson_r(afs["msl"], era20c["msl"], dim='time')
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["msl"], era20c_LN["msl"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["msl"], era20c_EN["msl"], dim='time')
        correlation_afs_time_list_LN.append(r_afs_time_LN)
        correlation_afs_time_list_EN.append(r_afs_time_EN)
        
    r_afs_space = xs.pearson_r(afs["msl"], era20c["msl"], dim=["lat", "lon"])
    rmse_afs_time = xs.rmse(afs["msl"], era20c["msl"], dim='time')
    rmse_afs_space = xs.rmse(afs["msl"], era20c["msl"], dim=["lat", "lon"])
    p_afs_time = xs.pearson_r_p_value(afs["msl"], era20c["msl"], dim="time")
    p_afs_space = xs.pearson_r_p_value(afs["msl"], era20c["msl"], dim=["lat", "lon"])
    correlation_afs_space_list.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)
    
    correlation_masked_afs_time_list=[]
    for member_index in np.arange(0,51):
        afs_single_member=afs.sel(member=member_index)
        outname="corr_afs_era20c_slp_"+season+"_"+str(member_index)
        corr_file=corr_2d_sign(data1=afs_single_member,data2=era20c,var1="msl",var2="msl",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        correlation_masked_afs_time_list.append(corr_file)
    concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    concat_mean=concat.mean(dim="member")
    concat_min=concat.min(dim="member")
    concat_max=concat.max(dim="member")
    correlation_masked_afs_time_list_season_max.append(concat_max)
    correlation_masked_afs_time_list_season_mean.append(concat_mean)
    correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_slp_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_slp_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=tcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")


        #print(ml["SLPm"].time.values)
        #print(era20c["SLPm"].time.values)
        r_ml_time = xs.pearson_r(ml["slp"], era20c["msl"], dim='time')
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["slp"], era20c_LN["msl"], dim='time')
            r_ml_time_EN = xs.pearson_r(ml_EN["slp"], era20c_EN["msl"], dim='time')
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
        r_ml_space = xs.pearson_r(ml["slp"], era20c["msl"], dim=["lat", "lon"])
        rmse_ml_time = xs.rmse(ml["slp"], era20c["msl"], dim='time')
        rmse_ml_space = xs.rmse(ml["slp"], era20c["msl"], dim=["lat", "lon"])
        p_ml_time = xs.pearson_r_p_value(ml["slp"], era20c["msl"], dim="time")
        p_ml_space = xs.pearson_r_p_value(ml["slp"], era20c["msl"], dim=["lat", "lon"])
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)#
    
        correlation_masked_ml_time_list=[]
        for member_index in np.arange(0,5):
            outname="corr_"+method+"era20c_slp_"+season+"_"+str(member_index)
            ml_single_member=ml.sel(member=member_index)
            corr_file=corr_2d_sign(data1=ml_single_member,data2=era20c,var1="slp",var2="msl",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
            correlation_masked_ml_time_list.append(corr_file)
        concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        concat_mean=concat.mean(dim="member")
        
        correlation_masked_ml_time_list_method_mean.append(concat_mean)
    


        
    correlation_ml_space_list.append(correlation_ml_space_list_methods)
    if season=="DJF":
        correlation_ml_time_list_EN.append(correlation_ml_time_list_methods_EN)
        correlation_ml_time_list_LN.append(correlation_ml_time_list_methods_LN)
    correlation_ml_time_list.append(correlation_ml_time_list_methods)
    rmse_ml_space_list.append(rmse_ml_space_list_methods)
    rmse_ml_time_list.append(rmse_ml_time_list_methods)
    p_ml_space_list.append(p_ml_space_list_methods)
    p_ml_time_list.append(p_ml_time_list_methods)
    correlation_masked_ml_time_list_season_mean.append(correlation_masked_ml_time_list_method_mean)


correlation_ml_time_list_EN_era20c=correlation_ml_time_list_EN    
correlation_ml_time_list_LN_era20c=correlation_ml_time_list_LN    

correlation_afs_time_list_EN_era20c=correlation_afs_time_list_EN    
correlation_afs_time_list_LN_era20c=correlation_afs_time_list_LN    
    
correlation_ml_space_list_era20c=correlation_ml_space_list
correlation_ml_time_list_era20c=correlation_ml_time_list
rmse_ml_space_list_era20c=rmse_ml_space_list
rmse_ml_time_list_era20c=rmse_ml_time_list
p_ml_space_list_era20c=p_ml_space_list
p_ml_time_list_era20c=p_ml_time_list
correlation_masked_afs_time_list_season_max_era20c=correlation_masked_afs_time_list_season_max # len == season
correlation_masked_afs_time_list_season_mean_era20c=correlation_masked_afs_time_list_season_mean
correlation_masked_afs_time_list_season_min_era20c=correlation_masked_afs_time_list_season_min

correlation_afs_space_list_era20c=correlation_afs_space_list
correlation_afs_time_list_era20c=correlation_afs_time_list
rmse_afs_space_list_era20c=rmse_afs_space_list
rmse_afs_time_list_era20c=rmse_afs_time_list
p_afs_space_list_era20c=p_afs_space_list
p_afs_time_list_era20c=p_afs_time_list
correlation_masked_ml_time_list_season_mean_era20c=correlation_masked_ml_time_list_season_mean # season*method

In [ ]:
seasons=["JJA","DJF"]
# this for slp
correlation_afs_space_list=[]
correlation_afs_time_list=[]
rmse_afs_space_list=[]
rmse_afs_time_list=[]
p_afs_space_list=[]
p_afs_time_list=[]

correlation_afs_time_list_LN=[]
correlation_afs_time_list_EN=[]
correlation_ml_time_list_LN=[]
correlation_ml_time_list_EN=[]


correlation_ml_space_list=[]
correlation_ml_time_list=[]
rmse_ml_space_list=[]
rmse_ml_time_list=[]
p_ml_space_list=[]
p_ml_time_list=[]

correlation_masked_afs_time_list_season_max=[]
correlation_masked_afs_time_list_season_mean=[]
correlation_masked_afs_time_list_season_min=[]


correlation_masked_ml_time_list_season_mean=[]
correlation_masked_ml_time_list_method_mean=[]




naming_list=[]
for season in seasons:
    
    correlation_ml_space_list_methods=[]
    correlation_ml_time_list_methods=[]
    rmse_ml_space_list_methods=[]
    rmse_ml_time_list_methods=[]
    p_ml_space_list_methods=[]
    p_ml_time_list_methods=[]
    correlation_ml_time_list_methods_LN=[]
    correlation_ml_time_list_methods_EN=[]
    
    print(asf20c_folder+"SLPmonthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc")
    afs=xr.open_mfdataset(asf20c_folder+"SLPmonthly_allyears_allseasons_M*_"+season+"_remap_anoms.nc",combine="nested",concat_dim="member")
    start_year=str(afs.forecast_time0.values[0])[0:4]
    end_year=str(afs.forecast_time0.values[-1])[0:4]
    
    print(tcr_folder+"slp_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr=xr.open_dataset(tcr_folder+"slp_mon.mean_"+season+"_"+month_dict[season]+"_lowlow_anoms.nc")
    tcr_time_long=tcr.time
    tcr=tcr.sel(time=slice(start_year,end_year))
    tcr=tcr.rename({'prmsl': 'slp'})
    
    afs=afs.rename({'forecast_time0': 'time'})
    afs=afs.rename({'MSL_GDS0_SFC': 'slp'})
    afs["time"]=tcr.time
    
    tcr_LN=tcr.sel(time=LN_nov_years_np,method="nearest")
    tcr_EN=tcr.sel(time=EN_nov_years_np,method="nearest")
    
    afs_LN=afs.sel(time=LN_nov_years_np,method="nearest")
    afs_EN=afs.sel(time=EN_nov_years_np,method="nearest")
    
    r_afs_time = xs.pearson_r(afs["slp"], tcr["slp"], dim='time')
    if season=="DJF":
        r_afs_time_LN = xs.pearson_r(afs_LN["slp"], tcr_LN["slp"], dim='time')
        r_afs_time_EN = xs.pearson_r(afs_EN["slp"], tcr_EN["slp"], dim='time')
        correlation_afs_time_list_LN.append(r_afs_time_LN)
        correlation_afs_time_list_EN.append(r_afs_time_EN)
    r_afs_space = xs.pearson_r(afs["slp"], tcr["slp"], dim=["lat", "lon"])
    rmse_afs_time = xs.rmse(afs["slp"], tcr["slp"], dim='time')
    rmse_afs_space = xs.rmse(afs["slp"], tcr["slp"], dim=["lat", "lon"])
    p_afs_time = xs.pearson_r_p_value(afs["slp"], tcr["slp"], dim="time")
    p_afs_space = xs.pearson_r_p_value(afs["slp"], tcr["slp"], dim=["lat", "lon"])
    correlation_afs_space_list.append(r_afs_space)
    correlation_afs_time_list.append(r_afs_time)
    rmse_afs_space_list.append(rmse_afs_space)
    rmse_afs_time_list.append(rmse_afs_time)
    p_afs_space_list.append(p_afs_space)
    p_afs_time_list.append(p_afs_time)
    
    correlation_masked_afs_time_list=[]
    for member_index in np.arange(0,51):
        afs_single_member=afs.sel(member=member_index)
        outname="corr_afs_tcr_slp_"+season+"_"+str(member_index)
        corr_file=corr_2d_sign(data1=afs_single_member,data2=tcr,var1="slp",var2="slp",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
        correlation_masked_afs_time_list.append(corr_file)
    concat=xr.concat(correlation_masked_afs_time_list,dim="member")
    concat_mean=concat.mean(dim="member")
    concat_min=concat.min(dim="member")
    concat_max=concat.max(dim="member")
    correlation_masked_afs_time_list_season_max.append(concat_max)
    correlation_masked_afs_time_list_season_mean.append(concat_mean)
    correlation_masked_afs_time_list_season_min.append(concat_min)
    
    correlation_masked_ml_time_list_method_mean=[]
    for method in methods_paper:
        print(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_slp_"+season+"_"+method+"m*_anoms.nc")
        ml=xr.open_mfdataset(prediction_folder+"best_modelanomnorm_mpicodaearlyctlrcp26_"+month_init_dict[season]+"_sst_slp_"+season+"_"+method+"m*_anoms.nc",combine="nested",concat_dim="member")
        ml["time"]=tcr_time_long
        ml=ml.sel(time=slice(start_year,end_year))
        
        ml_LN=ml.sel(time=LN_nov_years_np,method="nearest")
        ml_EN=ml.sel(time=EN_nov_years_np,method="nearest")

        
        r_ml_time = xs.pearson_r(ml["slp"], tcr["slp"], dim='time')
        if season=="DJF":
            r_ml_time_LN = xs.pearson_r(ml_LN["slp"], tcr_LN["slp"], dim='time')
            r_ml_time_EN = xs.pearson_r(ml_EN["slp"], tcr_EN["slp"], dim='time')
            correlation_ml_time_list_methods_LN.append(r_ml_time_LN)
            correlation_ml_time_list_methods_EN.append(r_ml_time_EN)
        r_ml_space = xs.pearson_r(ml["slp"], tcr["slp"], dim=["lat", "lon"])
        rmse_ml_time = xs.rmse(ml["slp"], tcr["slp"], dim='time')
        rmse_ml_space = xs.rmse(ml["slp"], tcr["slp"], dim=["lat", "lon"])
        p_ml_time = xs.pearson_r_p_value(ml["slp"], tcr["slp"], dim="time")
        p_ml_space = xs.pearson_r_p_value(ml["slp"], tcr["slp"], dim=["lat", "lon"])
        correlation_ml_space_list_methods.append(r_ml_space)
        correlation_ml_time_list_methods.append(r_ml_time)
        rmse_ml_space_list_methods.append(rmse_ml_space)
        rmse_ml_time_list_methods.append(rmse_ml_time)
        p_ml_space_list_methods.append(p_ml_space)
        p_ml_time_list_methods.append(p_ml_time)

        correlation_masked_ml_time_list=[]
        for member_index in np.arange(0,5):
            outname="corr_"+method+"tcr_slp_"+season+"_"+str(member_index)
            ml_single_member=ml.sel(member=member_index)
            corr_file=corr_2d_sign(data1=ml_single_member,data2=tcr,var1="slp",var2="slp",lon1="lon",lat1="lat",save_folder=folder_corr_fields,outname=outname,p=0.05)
            correlation_masked_ml_time_list.append(corr_file)
        concat=xr.concat(correlation_masked_ml_time_list,dim="member")
        concat_mean=concat.mean(dim="member")
        
        correlation_masked_ml_time_list_method_mean.append(concat_mean)
        
        
    correlation_ml_space_list.append(correlation_ml_space_list_methods)
    if season=="DJF":
        correlation_ml_time_list_EN.append(correlation_ml_time_list_methods_EN)
        correlation_ml_time_list_LN.append(correlation_ml_time_list_methods_LN)
    correlation_ml_time_list.append(correlation_ml_time_list_methods)
    rmse_ml_space_list.append(rmse_ml_space_list_methods)
    rmse_ml_time_list.append(rmse_ml_time_list_methods)
    p_ml_space_list.append(p_ml_space_list_methods)
    p_ml_time_list.append(p_ml_time_list_methods)
    correlation_masked_ml_time_list_season_mean.append(correlation_masked_ml_time_list_method_mean)

# Fig 8 / S14

In [ ]:
ml_time_LN_tcr=xr.concat([correlation_ml_time_list_LN[0][0],correlation_ml_time_list_LN[0][1],correlation_ml_time_list_LN[0][2],correlation_ml_time_list_LN[0][3],correlation_ml_time_list_LN[0][4],correlation_ml_time_list_LN[0][5]],dim="height")
ml_time_LN_era20c=xr.concat([correlation_ml_time_list_LN_era20c[0][0],correlation_ml_time_list_LN_era20c[0][1],correlation_ml_time_list_LN_era20c[0][2],correlation_ml_time_list_LN_era20c[0][3],correlation_ml_time_list_LN_era20c[0][4],correlation_ml_time_list_LN_era20c[0][5]],dim="height")
ml_time_EN_tcr=xr.concat([correlation_ml_time_list_EN[0][0],correlation_ml_time_list_EN[0][1],correlation_ml_time_list_EN[0][2],correlation_ml_time_list_EN[0][3],correlation_ml_time_list_EN[0][4],correlation_ml_time_list_EN[0][5]],dim="height")
ml_time_EN_era20c=xr.concat([correlation_ml_time_list_EN_era20c[0][0],correlation_ml_time_list_EN_era20c[0][1],correlation_ml_time_list_EN_era20c[0][2],correlation_ml_time_list_EN_era20c[0][3],correlation_ml_time_list_EN_era20c[0][4],correlation_ml_time_list_EN_era20c[0][5]],dim="height")

ds_merged_ens_tcr=xr.concat([ml_time_EN_tcr.mean(dim="height").mean(dim="member"),ml_time_LN_tcr.mean(dim="height").mean(dim="member"),ml_time_EN_tcr.mean(dim="height").mean(dim="member")-ml_time_LN_tcr.mean(dim="height").mean(dim="member"),correlation_afs_time_list_EN[0].mean(dim="member"),correlation_afs_time_list_LN[0].mean(dim="member"),correlation_afs_time_list_EN[0].mean(dim="member")-correlation_afs_time_list_LN[0].mean(dim="member")],dim="height")
ds_merged_ens_era20c=xr.concat([ml_time_EN_era20c.mean(dim="height").mean(dim="member"),ml_time_LN_era20c.mean(dim="height").mean(dim="member"),ml_time_EN_era20c.mean(dim="height").mean(dim="member")-ml_time_LN_era20c.mean(dim="height").mean(dim="member"),correlation_afs_time_list_EN_era20c[0].mean(dim="member"),correlation_afs_time_list_LN_era20c[0].mean(dim="member"),correlation_afs_time_list_EN_era20c[0].mean(dim="member")-correlation_afs_time_list_LN_era20c[0].mean(dim="member")],dim="height")

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(14, 5), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
INDEX=list(["(g)","(h)","(i)","(j)","(k)","(l)"])
axs=axs.flatten()

#Loop over all of the models
for i in range(len(INDEX)):
    tplot=ds_merged_ens_tcr[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(ds_merged_ens_tcr[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    #axs[i].set_title(methods_4_plots_publish[i], fontsize=5)

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.05, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"slp_corr_ENSO_tcr.pdf")
fig.savefig(output_folder+"slp_corr_ENSO_tcr.png")





In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(14, 5), dpi= 300,subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0, globe=None)})
INDEX=list(["(g)","(h)","(i)","(j)","(k)","(l)"])
axs=axs.flatten()

#Loop over all of the models
for i in range(len(INDEX)):
    tplot=ds_merged_ens_era20c[i,:,:].plot.pcolormesh(ax=axs[i],levels = 17, vmin=-1, vmax=1, transform=ccrs.PlateCarree(), cmap="seismic",add_colorbar=False)
    axs[i].set_title(INDEX[i],loc='left')#, fontsize=20
    axs[i].set_title(np.round(ds_merged_ens_era20c[i,:,:].mean().values,2),loc='right', fontsize=5)
    axs[i].coastlines()
    #axs[i].set_title(methods_4_plots_publish[i], fontsize=5)

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.2, right=0.8,
                    wspace=0.05, hspace=0.01)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.2, 0.2, 0.6, 0.02])



# Draw the colorbar
cbar=fig.colorbar(tplot, cax=cbar_ax,orientation='horizontal')
cbar.set_label('Pearson Corr Coeff', size=10) 




# Make it nice
plt.tight_layout()
plt.show()
fig.savefig(output_folder+"slp_corr_ENSO_era20c.pdf")
fig.savefig(output_folder+"slp_corr_ENSO_era20c.png")




